In [18]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [19]:
#read in dataset
traffic_acc = pd.read_csv("US_Accidents_Dec20_Updated.csv")
traffic_acc.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,2,2019-05-21 08:29:55,2019-05-21 09:29:40,34.808868,-82.269157,34.808868,-82.269157,0.0,Accident on Tanner Rd at Pennbrooke Ln.,...,False,False,False,False,False,False,Day,Day,Day,Day
1,A-2,2,2019-10-07 17:43:09,2019-10-07 19:42:50,35.090080,-80.745560,35.090080,-80.745560,0.0,Accident on Houston Branch Rd at Providence Br...,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-3,2,2020-12-13 21:53:00,2020-12-13 22:44:00,37.145730,-121.985052,37.165850,-121.988062,1.4,Stationary traffic on CA-17 from Summit Rd (CA...,...,False,False,False,False,False,False,Night,Night,Night,Night
3,A-4,2,2018-04-17 16:51:23,2018-04-17 17:50:46,39.110390,-119.773781,39.110390,-119.773781,0.0,Accident on US-395 Southbound at Topsy Ln.,...,False,False,False,False,True,False,Day,Day,Day,Day
4,A-5,3,2016-08-31 17:40:49,2016-08-31 18:10:49,26.102942,-80.265091,26.102942,-80.265091,0.0,Accident on I-595 Westbound at Exit 4 / Pine I...,...,False,False,False,False,True,False,Day,Day,Day,Day


In [20]:
#gather list of columns
for col_name in traffic_acc.columns:
    print(col_name)

ID
Severity
Start_Time
End_Time
Start_Lat
Start_Lng
End_Lat
End_Lng
Distance(mi)
Description
Number
Street
Side
City
County
State
Zipcode
Country
Timezone
Airport_Code
Weather_Timestamp
Temperature(F)
Wind_Chill(F)
Humidity(%)
Pressure(in)
Visibility(mi)
Wind_Direction
Wind_Speed(mph)
Precipitation(in)
Weather_Condition
Amenity
Bump
Crossing
Give_Way
Junction
No_Exit
Railway
Roundabout
Station
Stop
Traffic_Calming
Traffic_Signal
Turning_Loop
Sunrise_Sunset
Civil_Twilight
Nautical_Twilight
Astronomical_Twilight


```
At this point I think it's necessary to remove some of the columns that I don't intend to use. This will help the overall dataset load faster as well as narrow down the amount of data I need to analyze. Right now I intend on removing the columns:
1. Latitudes and Longitudes - I already have the states which is what I need for my analysis
2. Distance (mi) - shows the length of road affected by the accident
3. Number, Street, Side, County, Zipcode - Only interested in State but kept City incase I decide to analyze that at a granular level like L.A.
4. Timezone
5. Airport code
```

In [21]:
#Remove columns that I don't intend to use
traffic_acc = traffic_acc.drop(columns = {'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng',
                                         'Distance(mi)', 'Number', 'Street', 'Side', 'City','County', 'Zipcode', 
                                          'Country','Timezone','Airport_Code', 'Weather_Timestamp'})

#verify columns have been removed
for col_name in traffic_acc.columns:
    print(col_name)

ID
Severity
Start_Time
End_Time
Description
State
Temperature(F)
Wind_Chill(F)
Humidity(%)
Pressure(in)
Visibility(mi)
Wind_Direction
Wind_Speed(mph)
Precipitation(in)
Weather_Condition
Amenity
Bump
Crossing
Give_Way
Junction
No_Exit
Railway
Roundabout
Station
Stop
Traffic_Calming
Traffic_Signal
Turning_Loop
Sunrise_Sunset
Civil_Twilight
Nautical_Twilight
Astronomical_Twilight


In [22]:
#Grabbing number of individual records
len(traffic_acc.ID)

2906610

In [23]:
#Now I have the columns I want and number of records, I need to check for null entries
traffic_acc.isnull().sum()

ID                             0
Severity                       0
Start_Time                     0
End_Time                       0
Description                    0
State                          0
Temperature(F)             67224
Wind_Chill(F)            1183859
Humidity(%)                71270
Pressure(in)               56908
Visibility(mi)             72078
Wind_Direction             63474
Wind_Speed(mph)           307163
Precipitation(in)        1301326
Weather_Condition          71851
Amenity                        0
Bump                           0
Crossing                       0
Give_Way                       0
Junction                       0
No_Exit                        0
Railway                        0
Roundabout                     0
Station                        0
Stop                           0
Traffic_Calming                0
Traffic_Signal                 0
Turning_Loop                   0
Sunrise_Sunset               110
Civil_Twilight               110
Nautical_T

```
Looking at the null values throws a bit of a wrench into my thought process. I figured weather would have a big impact on the number of traffic accidents because I know it does locally. I have my total number of unique records, so some may be worth keeping, but I posted the question in the class chat for thoughts. I still have time of day and accident location type (for example railway) that will be beneficial to analyze.
```

In [24]:
#Posed the question to the class on whether I should keep the weather data with so many null values. 
#Decided to keep for now, but changing nan to 0.
traffic_acc['Temperature(F)'] = traffic_acc['Temperature(F)'].fillna(0)
traffic_acc['Wind_Chill(F)'] = traffic_acc['Wind_Chill(F)'].fillna(0)
traffic_acc['Humidity(%)'] = traffic_acc['Humidity(%)'].fillna(0)
traffic_acc['Pressure(in)'] = traffic_acc['Pressure(in)'].fillna(0)
traffic_acc['Visibility(mi)'] = traffic_acc['Visibility(mi)'].fillna(0)
traffic_acc['Wind_Direction'] = traffic_acc['Wind_Direction'].fillna(0)
traffic_acc['Wind_Speed(mph)'] = traffic_acc['Wind_Speed(mph)'].fillna(0)
traffic_acc['Precipitation(in)'] = traffic_acc['Precipitation(in)'].fillna(0)
traffic_acc['Weather_Condition'] = traffic_acc['Weather_Condition'].fillna(0)

#Checking that it worked
traffic_acc.isnull().sum()

ID                         0
Severity                   0
Start_Time                 0
End_Time                   0
Description                0
State                      0
Temperature(F)             0
Wind_Chill(F)              0
Humidity(%)                0
Pressure(in)               0
Visibility(mi)             0
Wind_Direction             0
Wind_Speed(mph)            0
Precipitation(in)          0
Weather_Condition          0
Amenity                    0
Bump                       0
Crossing                   0
Give_Way                   0
Junction                   0
No_Exit                    0
Railway                    0
Roundabout                 0
Station                    0
Stop                       0
Traffic_Calming            0
Traffic_Signal             0
Turning_Loop               0
Sunrise_Sunset           110
Civil_Twilight           110
Nautical_Twilight        110
Astronomical_Twilight    110
dtype: int64

In [25]:
#Now changing the true/false values to 1/0
traffic_acc['Amenity'] = traffic_acc['Amenity'].astype(int)
traffic_acc['Bump'] = traffic_acc['Bump'].astype(int)
traffic_acc['Crossing'] = traffic_acc['Crossing'].astype(int)
traffic_acc['Give_Way'] = traffic_acc['Give_Way'].astype(int)
traffic_acc['Junction'] = traffic_acc['Junction'].astype(int)
traffic_acc['No_Exit'] = traffic_acc['No_Exit'].astype(int)
traffic_acc['Railway'] = traffic_acc['Railway'].astype(int)
traffic_acc['Roundabout'] = traffic_acc['Roundabout'].astype(int)
traffic_acc['Station'] = traffic_acc['Station'].astype(int)
traffic_acc['Stop'] = traffic_acc['Stop'].astype(int)
traffic_acc['Traffic_Calming'] = traffic_acc['Traffic_Calming'].astype(int)
traffic_acc['Traffic_Signal'] = traffic_acc['Traffic_Signal'].astype(int)
traffic_acc['Turning_Loop'] = traffic_acc['Turning_Loop'].astype(int)

#Check that it worked
traffic_acc.head()

,ID,Severity,Start_Time,End_Time,Description,State,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,2,2019-05-21 08:29:55,2019-05-21 09:29:40,Accident on Tanner Rd at Pennbrooke Ln.,SC,76.0,76.0,52.0,28.91,...,0,0,0,0,0,0,Day,Day,Day,Day
1,A-2,2,2019-10-07 17:43:09,2019-10-07 19:42:50,Accident on Houston Branch Rd at Providence Br...,NC,76.0,76.0,62.0,29.30,...,0,0,0,0,0,0,Day,Day,Day,Day
2,A-3,2,2020-12-13 21:53:00,2020-12-13 22:44:00,Stationary traffic on CA-17 from Summit Rd (CA...,CA,51.0,51.0,80.0,30.17,...,0,0,0,0,0,0,Night,Night,Night,Night
3,A-4,2,2018-04-17 16:51:23,2018-04-17 17:50:46,Accident on US-395 Southbound at Topsy Ln.,NV,53.6,0.0,16.0,30.16,...,0,0,0,0,1,0,Day,Day,Day,Day
4,A-5,3,2016-08-31 17:40:49,2016-08-31 18:10:49,Accident on I-595 Westbound at Exit 4 / Pine I...,FL,84.2,0.0,84.0,29.92,...,0,0,0,0,1,0,Day,Day,Day,Day


```
The last missing values that I need to attend to are the ones for time of day. First, I need to figure out what the different types mean, and then I'll update the nan values to 0. Here are the descriptions of each time of day from kaggle:

Sunrise_Sunset = Period of day based on sunrise or sunset
Civil_Twilight = Period of day based on civil twilight
Nautical_Twilight = Period of day based on nautical twilight
Astronomical_Twilight = Period of day based on astronomical twilight

"Civil twilight is the brightest of the 3 twilight phases. The Sun is just below the horizon, so there is generally enough natural light to carry out most outdoor activities." https://www.timeanddate.com/astronomy/civil-twilight.html

"Nautical twilight is the second twilight phase. Both the horizon and the brighter stars are usually visible at this time, makig it possible to navigate at sea." https://www.timeanddate.com/astronomy/nautical-twilight.html

"Astronomical twilight is the darkest of the 3 twilight phases. It is the earliest state of dawn in the morning and the last stage of dusk in the evening." https://www.timeanddate.com/astronomy/astronomical-twilight.html
```

In [26]:
#Remove missing values from time of day and replace with 0
traffic_acc['Sunrise_Sunset'] = traffic_acc['Sunrise_Sunset'].fillna(0)
traffic_acc['Civil_Twilight'] = traffic_acc['Civil_Twilight'].fillna(0)
traffic_acc['Nautical_Twilight'] = traffic_acc['Nautical_Twilight'].fillna(0)
traffic_acc['Astronomical_Twilight'] = traffic_acc['Astronomical_Twilight'].fillna(0)

#Check I have no more missing values
traffic_acc.isnull().sum()

ID                       0
Severity                 0
Start_Time               0
End_Time                 0
Description              0
State                    0
Temperature(F)           0
Wind_Chill(F)            0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Wind_Direction           0
Wind_Speed(mph)          0
Precipitation(in)        0
Weather_Condition        0
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
Station                  0
Stop                     0
Traffic_Calming          0
Traffic_Signal           0
Turning_Loop             0
Sunrise_Sunset           0
Civil_Twilight           0
Nautical_Twilight        0
Astronomical_Twilight    0
dtype: int64

```
The last thing I want to do for clean-up on this dataset before moving on to visuals and analyzing, is to separate my date/time columns. Really I only need the month and year for what I want to look at since the time of day variables can give me day or night. 
```

In [27]:
#On the date/time I really only want the year and month, so creating new columns for those out of Start_Time
traffic_acc['Start_Time'] = pd.to_datetime(traffic_acc['Start_Time'], format = '%Y-%m-%d')
traffic_acc['Year'] = pd.DatetimeIndex(traffic_acc['Start_Time']).year
traffic_acc['Month'] = pd.DatetimeIndex(traffic_acc['Start_Time']).month

#Test to see if I got the new columns
traffic_acc.head()

,ID,Severity,Start_Time,End_Time,Description,State,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month
0,A-1,2,2019-05-21 08:29:55,2019-05-21 09:29:40,Accident on Tanner Rd at Pennbrooke Ln.,SC,76.0,76.0,52.0,28.91,...,0,0,0,0,Day,Day,Day,Day,2019,5
1,A-2,2,2019-10-07 17:43:09,2019-10-07 19:42:50,Accident on Houston Branch Rd at Providence Br...,NC,76.0,76.0,62.0,29.30,...,0,0,0,0,Day,Day,Day,Day,2019,10
2,A-3,2,2020-12-13 21:53:00,2020-12-13 22:44:00,Stationary traffic on CA-17 from Summit Rd (CA...,CA,51.0,51.0,80.0,30.17,...,0,0,0,0,Night,Night,Night,Night,2020,12
3,A-4,2,2018-04-17 16:51:23,2018-04-17 17:50:46,Accident on US-395 Southbound at Topsy Ln.,NV,53.6,0.0,16.0,30.16,...,0,0,1,0,Day,Day,Day,Day,2018,4
4,A-5,3,2016-08-31 17:40:49,2016-08-31 18:10:49,Accident on I-595 Westbound at Exit 4 / Pine I...,FL,84.2,0.0,84.0,29.92,...,0,0,1,0,Day,Day,Day,Day,2016,8


In [28]:
#Now that I have the Year and Month columns I don't need Start_Time or End_Time so I'm going to remove those
traffic_acc = traffic_acc.drop(columns = {'Start_Time', 'End_Time'})
traffic_acc.head()

,ID,Severity,Description,State,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month
0,A-1,2,Accident on Tanner Rd at Pennbrooke Ln.,SC,76.0,76.0,52.0,28.91,10.0,N,...,0,0,0,0,Day,Day,Day,Day,2019,5
1,A-2,2,Accident on Houston Branch Rd at Providence Br...,NC,76.0,76.0,62.0,29.30,10.0,VAR,...,0,0,0,0,Day,Day,Day,Day,2019,10
2,A-3,2,Stationary traffic on CA-17 from Summit Rd (CA...,CA,51.0,51.0,80.0,30.17,10.0,W,...,0,0,0,0,Night,Night,Night,Night,2020,12
3,A-4,2,Accident on US-395 Southbound at Topsy Ln.,NV,53.6,0.0,16.0,30.16,10.0,SSW,...,0,0,1,0,Day,Day,Day,Day,2018,4
4,A-5,3,Accident on I-595 Westbound at Exit 4 / Pine I...,FL,84.2,0.0,84.0,29.92,10.0,SSE,...,0,0,1,0,Day,Day,Day,Day,2016,8


In [30]:
#having problems changing values, so saving off a csv then updating in the csv itself
traffic_acc.to_csv('traffic_accidents.csv')